In [ ]:
!pip install python-Levenshtein
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from collections import Counter
import pandas as pd
import Levenshtein
from tqdm import tqdm

In [ ]:
df = pd.read_csv('tg_data_labeled_ru_ua.csv')

<ipython-input-2-5a54fd5d3810>:1: DtypeWarning: Columns (12,20,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('tg_data_labeled_ru_ua.csv')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77780 entries, 0 to 77779
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Unnamed: 0                              77780 non-null  int64  
 1   msg_link                                77780 non-null  object 
 2   message                                 77780 non-null  object 
 3   views                                   77780 non-null  int64  
 4   cluster_label_dbscan                    77780 non-null  int64  
 5   weapon                                  13130 non-null  object 
 6   location                                49709 non-null  object 
 7   org                                     30833 non-null  object 
 8   person                                  13722 non-null  object 
 9   repeated_text                           22785 non-null  object 
 10  Реклама                                 36380 non-null  fl

In [ ]:

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # We leave only letters and numbers
    text = text.lower()
    return text

df['clean_message'] = df['message'].apply(clean_text)

def remove_emojis(text):
    emoji_pattern = re.compile(
        "[\U00010000-\U0010ffff]", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['clean_message'] = df['clean_message'].apply(remove_emojis)
df['clean_message']

df.to_csv('Cleaned_df.csv')
df.head()


,Unnamed: 0,msg_link,message,views,cluster_label_dbscan,weapon,location,org,person,repeated_text,...,Тема,Критика російського,Що критикують,Персоналії обороноздатності РФ,Цікавить СОУ,Що саме може зацікавити СОУ?,Досвід,Який саме досвід?,Обговорення,clean_message
0,0,GorInformburo/40989,Генсек НАТО:. Что касается восстановления Укра...,98,0,NaN,Украины;Украина;Украине,НАТО,NaN,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,,генсек нато что касается восстановления украин...
1,1,Cbpub/50635,Украину не будет смысла восстанавливать в случ...,248103,0,NaN,Украину;Украины;Украина;Украине,НАТО,NaN,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,украину не будет смысла восстанавливать в случ...
2,2,izvestia/171328,Украину не будет смысла восстанавливать в случ...,14623,0,NaN,Украину;Украины;Украина;Украине,НАТО,Йенс Столтенберг,Подписаться на IZ.RU,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,украину не будет смысла восстанавливать в случ...
3,3,donbassrussiazvo/10896,"📣 Генсек НАТО считает, что Украину не будет см...",302,0,NaN,Украину;Украины;Украина;Украине,НАТО,Столтенберг,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,генсек нато считает что украину не будет смыс...
4,4,RVvoenkor/68075,"‼️🇺🇦🏴‍☠Генсек НАТО считает, что Украину не буд...",510610,0,NaN,Украину;Украины;Украина;Украине,НАТО,Столтенберг,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,генсек нато считает что украину не будет смысл...


In [ ]:

# df = pd.DataFrame({'clean_message': ['текст 1', 'текст 2', 'текст 3', ...]})

# df_1 = df.iloc[:5000]
# df_2 = df.iloc[5000:10000]
df_3 = df.iloc[10000:15000]
df_4 = df.iloc[15000:20000]
df_5 = df.iloc[20000:25000]
# df_6 = df.iloc[25000:30000]
# df_7 = df.iloc[30000:35000]
# df_8 = df.iloc[35000:40000]
# df_9 = df.iloc[40000:45000]
# df_10 = df.iloc[45000:50000]
# df_11 = df.iloc[50000:55000]
# df_12 = df.iloc[55000:60000]
# df_13 = df.iloc[60000:65000]
# df_14 = df.iloc[65000:70000]
# df_15 = df.iloc[70000:]

def compute_levenshtein(df_part):
    df_part = df_part.reset_index(drop=True)
    levenshtein_matrix = []

    # Turn on the progress bar for the outer cycle (by i)
    for i in tqdm(range(len(df_part)), desc="Обчислення відстаней", ncols=100):
        row = []
        for j in range(len(df_part)):
            dist = Levenshtein.distance(df_part['clean_message'][i], df_part['clean_message'][j])
            row.append(dist)
        levenshtein_matrix.append(row)
    levenshtein_df = pd.DataFrame(levenshtein_matrix, columns=range(len(df_part)), index=range(len(df_part)))
    return levenshtein_df


# Обчислюємо відстані для кожної частини і зберігаємо результат
# levenshtein_df_1 = compute_levenshtein(df_1)
# levenshtein_df_2 = compute_levenshtein(df_2)
# levenshtein_df_3 = compute_levenshtein(df_3)
levenshtein_df_4 = compute_levenshtein(df_4)
# levenshtein_df_5 = compute_levenshtein(df_5)
# levenshtein_df_6 = compute_levenshtein(df_6)
# levenshtein_df_7 = compute_levenshtein(df_7)
# levenshtein_df_8 = compute_levenshtein(df_8)
# levenshtein_df_9 = compute_levenshtein(df_9)
# levenshtein_df_10 = compute_levenshtein(df_10)
# levenshtein_df_11 = compute_levenshtein(df_11)
# levenshtein_df_12 = compute_levenshtein(df_12)
# levenshtein_df_13 = compute_levenshtein(df_13)
# levenshtein_df_14 = compute_levenshtein(df_14)
# levenshtein_df_15 = compute_levenshtein(df_15)

# Виводимо одну з матриць для перевірки результатів

Обчислення відстаней:   0%|▏                                      | 19/5000 [00:03<15:18,  5.43it/s]


KeyboardInterrupt: 

In [ ]:
# df_1 = df.iloc[:5000]
# df_2 = df.iloc[5000:10000]
df_3 = df.iloc[10000:15000]
df_4 = df.iloc[15000:20000]
df_5 = df.iloc[20000:25000]
df_6 = df.iloc[25000:30000]
df_7 = df.iloc[30000:35000]
df_8 = df.iloc[35000:40000]
df_9 = df.iloc[40000:45000]
df_10 = df.iloc[45000:50000]
df_11 = df.iloc[50000:55000]
df_12 = df.iloc[55000:60000]
df_13 = df.iloc[60000:65000]
df_14 = df.iloc[65000:70000]
df_15 = df.iloc[70000:]

In [ ]:
levenshtein_df_15 = pd.read_csv("/content/levenshtein_df_15.csv")

In [ ]:
levenshtein_df_15 = levenshtein_df_15.drop(columns=['Unnamed: 0'])

In [ ]:
to_remove = set()
threshold = 85
for i in range(len(levenshtein_df_15)):
    for j in range(i + 1, len(levenshtein_df_15)):
        if levenshtein_df_15.iloc[i, j] <= threshold:  # Використовуємо .iloc для доступу
            to_remove.add(j)  # Додаємо індекс, який схожий на i


In [ ]:
df_15 = df_15.reset_index()
filtered_df = df_15.drop(index=to_remove)

In [ ]:
filtered_df.to_csv("df_15.csv")